# Fake DDOS generator

This notebook generates fake DDoS traffic every 0.01 seconds.

In [1]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python requests # type: ignore

In [2]:
ddos_event = """{"visitor_platform": "mobile","ts_ingest": 1515819844345,"article_title": "Cercanías San Sebastián","visitor_country": "BE","visitor_page_timer": 0,"visitor_os": "ios","article": "https://en.wikipedia.org/wiki/Cercan%C3%ADas_San_Sebasti%C3%A1n","visitor_page_height": 0,"visitor_browser": "unknown"}"""

In [3]:
# Send the ratings to the endpoint in order to test your implementation of the endpoint.
# SENDMODE = "http"

# Send the ratings straight to Kafka in order to simulate a working endpoint.
SENDMODE = "kafka"

In [ ]:
from datetime import datetime
import itertools
import json
import time

import requests
from kafka import KafkaProducer
from kafka.errors import KafkaError
from IPython.display import clear_output

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

orig_start = 0
replay_start = time.time() * 1000

counter = 0
i = 0

print("Simulating ddos traffic...")

ddos_event_dict = json.loads(ddos_event)

for _ in itertools.repeat(None, 30):
    ddos_event_dict['ts_ingest'] = round(time.time() * 1000)
    if SENDMODE == "kafka":        
        future = producer.send('clicks', ddos_event_dict)
        try:
            record_metadata = future.get(timeout=10)
        except KafkaError:
            print("exception")
            pass
    else:
        requests.post("http://localhost:5000/clicks", data=json.dumps(ddos_event_dict).encode("utf-8"), headers={'Content-Type':'application/json'})

    print("✓")
    if i > 1000:
        clear_output()
        i = 0
    i = i+1
    time.sleep(0.001)
